In [5]:
# --- 환경 준비 (한 번만) ---
# exiftool은 OS에 설치되어 있어야 합니다. (Ubuntu: sudo apt-get install exiftool)
# pandas만 없으면 아래 실행
# !pip install -q pandas

import subprocess, json, pandas as pd
from pathlib import Path

# --- 경로 설정 ---
BASE_DIR   = Path("/data1/project/yeonu/capstone")
PHOTO_DIR  = BASE_DIR / "data" / "photo"
OUT_CSV    = BASE_DIR / "data" / "photo_meta.csv"

# --- 대상 파일 수집 ---
exts = {".jpg", ".jpeg", ".png", ".tif", ".tiff", ".heic", ".heif", ".cr2", ".nef", ".arw", ".dng"}
files = sorted([str(p) for p in PHOTO_DIR.iterdir() if p.suffix.lower() in exts])

if not files:
    raise FileNotFoundError(f"이미지 파일이 없습니다: {PHOTO_DIR}")

# --- 추출할 필드 ---
fields = [
    "SourceFile", "FileName", "Directory", "Make", "Model",
    "DateTimeOriginal", "CreateDate", "ModifyDate",
    "ExposureTime", "FNumber", "ISO",
    "GPSLatitude", "GPSLongitude", "GPSAltitude"
]

# --- exiftool 실행 (-n: 숫자형, -json: JSON 출력) ---
cmd = ["exiftool", "-json", "-n"] + [f"-{k}" for k in fields] + files
out = subprocess.check_output(cmd)
data = json.loads(out)

# --- DataFrame 구성 ---
df = pd.DataFrame(data)

# 보기 좋은 캡처 시간 컬럼 추가 (없으면 CreateDate 대체)
dto = df.get("DateTimeOriginal")
if dto is not None:
    df.insert(4, "CapturedTime", dto.fillna(df.get("CreateDate")))
else:
    # DateTimeOriginal 자체가 없을 수도 있음
    df.insert(4, "CapturedTime", df.get("CreateDate"))

# 위경도 컬럼명 정리
df.rename(columns={"GPSLatitude": "Latitude", "GPSLongitude": "Longitude"}, inplace=True)

# --- 저장 ---
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(OUT_CSV, index=False, encoding="utf-8")
print(f"저장 완료: {OUT_CSV} | 행수: {len(df)}")

# 미리보기
df.head(10)


저장 완료: /data1/project/yeonu/capstone/data/photo_meta.csv | 행수: 10


   10 image files read


,SourceFile,FileName,Directory,Make,CapturedTime,Model,DateTimeOriginal,CreateDate,ModifyDate,ExposureTime,FNumber,ISO,Latitude,Longitude,GPSAltitude
0,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20250929_011857155.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:09:27 20:12:55,iPhone 11,2025:09:27 20:12:55,2025:09:27 20:12:55,2025:09:27 20:12:55,0.125000,1.8,640,37.537031,126.945808,62.511064
1,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20250929_011857155_01.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:09:26 22:15:31,iPhone 11,2025:09:26 22:15:31,2025:09:26 22:15:31,2025:09:26 22:15:31,0.066667,1.8,1000,37.558444,127.000083,86.352747
2,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20250929_011857155_02.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:09:26 19:55:37,iPhone 11,2025:09:26 19:55:37,2025:09:26 19:55:37,2025:09:26 19:55:37,0.033333,1.8,800,37.556553,127.001106,77.399855
3,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20250929_011857155_03.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:09:26 18:51:49,iPhone 11,2025:09:26 18:51:49,2025:09:26 18:51:49,2025:09:26 18:51:49,0.500000,1.8,1000,37.559722,126.999594,78.044911
4,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20251013_201618088.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:09:29 13:10:30,iPhone 11,2025:09:29 13:10:30,2025:09:29 13:10:30,2025:09:29 13:10:30,0.008333,1.8,80,37.556975,127.003028,66.403985
5,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20251013_201618088_02.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:10:01 13:19:29,iPhone 11,2025:10:01 13:19:29,2025:10:01 13:19:29,2025:10:01 13:19:29,0.008333,1.8,64,37.562481,127.003928,41.175348
6,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20251013_201618088_03.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:10:01 13:40:49,iPhone 11,2025:10:01 13:40:49,2025:10:01 13:40:49,2025:10:01 13:40:49,0.000493,1.8,32,37.562497,127.004411,44.903452
7,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20251013_201618088_04.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:10:02 19:43:52,iPhone 11,2025:10:02 19:43:52,2025:10:02 19:43:52,2025:10:02 19:43:52,0.016667,2.4,640,37.540036,126.943778,22.070556
8,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20251013_201618088_05.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:10:11 16:56:37,iPhone 11,2025:10:11 16:56:37,2025:10:11 16:56:37,2025:10:11 16:56:37,0.025000,1.8,640,37.576672,126.975197,46.970984
9,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20251013_201618088_06.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:10:12 16:33:26,iPhone 11,2025:10:12 16:33:26,2025:10:12 16:33:26,2025:10:12 16:33:26,0.025000,1.8,400,37.540072,126.948853,30.648564
